# Game Review Analysis Project Documentation

## Final project for text analysis of unstructured sources 

### Students
- Guilherme Fontana Louro
- Ricardo Ribeiro Rodrigues
- William Kenzo
- Natália Queiroz Carreras

## Project Overview
This project aims to analyze Steam game reviews using Natural Language Processing (NLP) techniques to identify key words and patterns in positive and negative reviews. The analysis provides insights into what aspects of games players appreciate or criticize the most.

## Main Components

### 1. Data Collection
- Web scraping of Steam reviews using Selenium
- Automated scrolling to collect a large number of reviews
- Extraction of review text, recommendation status, and helpful votes
- Support for multiple languages (e.g., Brazilian Portuguese)

### 2. Data Preprocessing
- Cleaning of review text (removing dates, extra spaces)
- Text normalization and preparation for NLP analysis
- Handling of special characters and formatting

### 3. NLP Analysis
- Keyword extraction and frequency analysis
- Identification of common themes in positive and negative reviews
- Analysis with LLM of most relevant reviews

### 4. Visualization and Insights
- Frequency distributions for words based on its classification
- Comparative analysis between positive and negative reviews

## Results and Findings
- Identification of key factors influencing player satisfaction
- Common themes in positive and negative reviews
- Qualitative analysis on the most relavant reviews using LLM

## Usage Instructions
1. Ensure all required Python packages are installed (see requirements.txt)
2. Configure the game ID in the scraping script
3. Run the data collection process
4. Execute the analysis notebooks in sequence
5. Review the generated visualizations and insights

# Data collection

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import re

In [3]:
service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)

#### How to Get the Game's Review URL
To get the URL for a game's reviews, search for the game on Steam, enter its page, and get the ID that appears after `/app/`

Example:
Steam URL for FIFA: `https://store.steampowered.com/app/1506830/FIFA_22/`

The ID is `1506830`, and the link for reviews is: https://steamcommunity.com/app/1506830/reviews/?browsefilter=toprated

Scrapping "EA SPORTS FC™ 25" game reviews on steam

In [ ]:
# Replace with your game's review URL
URL = "https://steamcommunity.com/app/2669320/reviews/?browsefilter=toprated"

# URL += "&filterLanguage=brazilian" # To use pt-br filter

reviews = []
seen_reviews = set()
# Variable to keep track of the last seen card, to avoid duplicates

def get_review_data(scroll_count, last_seen_card):
    review_cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")
    print(f"Found {len(review_cards)} review cards on scroll {scroll_count + 1}")

    for card in review_cards[last_seen_card:]:
        try:
            # Get recommendation (Recommended or Not Recommended)
            recommendation = card.find_element(By.CLASS_NAME, "title").text.strip()

            review_text = card.find_element(By.CLASS_NAME, "apphub_CardTextContent").text.strip()

            # Relevance
            relevance = card.find_element(By.CLASS_NAME, "found_helpful").text.strip()
            number_of_helpful = re.search(r"(\d[\d,]*) people found this review helpful", relevance)
            extracted_number = 0
            if number_of_helpful:
                extracted_number = int(number_of_helpful.group(1).replace(",", ""))
            

            if review_text and review_text not in seen_reviews:
                seen_reviews.add(review_text)
                reviews.append({"text": review_text, "recommendation": recommendation, "helpful": extracted_number})
        except Exception as e:
            print("Error processing a review card:", e)
    
    return len(review_cards)


driver.get(URL)
time.sleep(3)
    
SCROLL_PAUSE = 1
# Number of times to scroll down the page, use inf to scroll until the end
# N_SCROLLS = float("inf")
N_SCROLLS = 40
last_height = driver.execute_script("return document.body.scrollHeight")
last_seen_card = 0


print("Scrolling and collecting reviews...")
for i in range(N_SCROLLS):
    # Scroll to the bottom of the page, only if not the first scroll
    if i > 0:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No more new content loaded. Stopping scroll.")
            break
        last_height = new_height
    
    last_seen_card = get_review_data(i, last_seen_card)

print(f"Collected {len(reviews)} reviews.")   

driver.quit()

reviews[0:10]

Scrolling and collecting reviews...
Found 10 review cards on scroll 1
Found 20 review cards on scroll 2
Found 30 review cards on scroll 3
Found 40 review cards on scroll 4
Found 50 review cards on scroll 5
Found 60 review cards on scroll 6
Found 70 review cards on scroll 7
Found 80 review cards on scroll 8
Found 90 review cards on scroll 9
Found 100 review cards on scroll 10
Found 110 review cards on scroll 11
Found 120 review cards on scroll 12
Found 130 review cards on scroll 13
Found 140 review cards on scroll 14
Found 150 review cards on scroll 15
Found 160 review cards on scroll 16
Found 170 review cards on scroll 17
Found 180 review cards on scroll 18
Found 190 review cards on scroll 19
Found 200 review cards on scroll 20
Found 210 review cards on scroll 21
Found 220 review cards on scroll 22
Found 230 review cards on scroll 23
Found 240 review cards on scroll 24
Found 250 review cards on scroll 25
Found 260 review cards on scroll 26
Found 270 review cards on scroll 27
Found 280 

[{'text': "Posted: 19 November, 2024\nTL:DR - This game is a psychologically manipulative Skinner box. It is designed to be addictive, and to get as much money out of you as possible. Scripting, implemented to encourage FUT players to spend more money, is in every mode and makes every game an unsatisfying and empty experience. Considering a core demographic for this game is children, the income-focused manipulation displayed here is repugnant.\n\nDDA (dynamic difficulty adjustment) or scipting is just ridiculous now. In games against EA's garbage AI, if the game wants your opponent to score, they'll respond to your controller inputs instantly, not the actions of the player you're controlling. Easy tackles with world class defenders will either miss, bounce the ball to another opponent, just outright phase through the ball or even the damn player! Your keeper will do a Matrix-esque dodge of a shot coming directly at them or will spill an easy catch directly into their striker. The most 

## Cleaning

### Removing the date from the review

Pattern `"Posted: <day> <month>[, <year>]\n"`

### Remove aditional spaces

Remove extra spaces - This makes it easier to read the resulting CSV, cuts characters that are not usefull from the output and needs to be done anyway for the data analysis.

### Remove the hearts that censor bad words

In [ ]:
def clean_text(text):
    text = re.sub(r"\s*♥+\s*", " ", text)  # Remove hearts that censor bad words on steam
    text = re.sub(r'\s+', ' ', text)  
    return text.strip()

def remove_posted_date(text):
    lines = text.splitlines()
    if lines and lines[0].startswith("Posted:"):
        return "\n".join(lines[1:]).strip()
    return text.strip()

for review in reviews:
    review["text"] = remove_posted_date(review["text"])
    review["text"] = clean_text(review["text"])

reviews[0:10]


[{'text': "TL:DR - This game is a psychologically manipulative Skinner box. It is designed to be addictive, and to get as much money out of you as possible. Scripting, implemented to encourage FUT players to spend more money, is in every mode and makes every game an unsatisfying and empty experience. Considering a core demographic for this game is children, the income-focused manipulation displayed here is repugnant. DDA (dynamic difficulty adjustment) or scipting is just ridiculous now. In games against EA's garbage AI, if the game wants your opponent to score, they'll respond to your controller inputs instantly, not the actions of the player you're controlling. Easy tackles with world class defenders will either miss, bounce the ball to another opponent, just outright phase through the ball or even the damn player! Your keeper will do a Matrix-esque dodge of a shot coming directly at them or will spill an easy catch directly into their striker. The most outrageous though, is player s

In [ ]:

with open("steam_reviews.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["Recommendation", "Review", "Helpful"])
    writer.writeheader()
    for review in reviews:
        writer.writerow({
            "Recommendation": review["recommendation"],
            "Review": review["text"],
            "Helpful": review["helpful"]
        })


print(f"Scraped {len(reviews)} reviews.")

Scraped 400 reviews.


## Data Analysis

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [49]:
nltk.download('stopwords')
ENGLISH_STOP_WORDS = list(stopwords.words('english'))
MEANING_LESS_WORDS = ["game", "play", "like", "good", "bad", "great", "fun", "time", "best", "worst", "dont", "fifa", "player", "players"]
ENGLISH_STOP_WORDS.extend(MEANING_LESS_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ricardorr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
extracted = pd.read_csv("steam_reviews.csv")
extracted.head(5)

,Recommendation,Review,Helpful
0,Not Recommended,TL:DR - This game is a psychologically manipul...,799
1,Not Recommended,"Worst Fifa ever, terrible passing, shooting, l...",710
2,Not Recommended,"gameplay, servers, full of bugs and problems. ...",292
3,Not Recommended,Year after year EA finds ways to make the game...,457
4,Not Recommended,I purchased this game 2 weeks ago and I still ...,436


In [51]:
def is_valid_word(word):
    # remove repeated letters, short words
    if len(word) <= 3:
        return False
    if re.search(r'(.)\1{2,}', word):
        return False
    if not re.match(r'^[a-zA-Z]+$', word):  # only letters
        return False
    return True

def clean_text(text):
    words = text.lower().split()
    return ' '.join([word for word in words if is_valid_word(word)])

In [52]:
# Translate non-ASCII characters to ASCII
extracted['CleanedReview'] = extracted['Review'].apply(unidecode)
# Remove numbers and punctuation
extracted['CleanedReview'] = extracted['CleanedReview'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
extracted['CleanedReview'] = extracted['CleanedReview'].apply(clean_text)

extracted.head(5)

,Recommendation,Review,Helpful,CleanedReview
0,Not Recommended,TL:DR - This game is a psychologically manipul...,799,tldr this game psychologically manipulative sk...
1,Not Recommended,"Worst Fifa ever, terrible passing, shooting, l...",710,worst fifa ever terrible passing shooting inpu...
2,Not Recommended,"gameplay, servers, full of bugs and problems. ...",292,gameplay servers full bugs problems buying thi...
3,Not Recommended,Year after year EA finds ways to make the game...,457,year after year finds ways make game worse esp...
4,Not Recommended,I purchased this game 2 weeks ago and I still ...,436,purchased this game weeks still able play sing...


In [67]:
satisfied = extracted[extracted["Recommendation"] == 'Recommended']
not_satisfied = extracted[extracted['Recommendation'] == 'Not Recommended']
len(satisfied), len(not_satisfied)

(83, 317)

## Model used - Count vectorizer + logistic regression

Predicts how much each word (or ngram) relates to being the target variable, assigning weights to each of the tokens, those weights can than be used to measure how negative (or positive) a word is.

In [54]:
X = extracted["CleanedReview"].to_numpy()
Y = extracted["Recommendation"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)

classifier = Pipeline(
    [
        ('vectorizer', CountVectorizer(stop_words=ENGLISH_STOP_WORDS, binary=True, ngram_range=(1, 2))),
        ('log_regressor', LogisticRegression(penalty='l2', solver='saga', max_iter=10_000))
    ]
)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_pred, y_test)
f"Training accuracy: {acc} - {len(X_train)} samples, {len(X_test)} test samples"

'Training accuracy: 0.8333333333333334 - 340 samples, 60 test samples'

In [30]:
classifier.classes_
# 0 = Not Recommended; 1 = Recommended

array(['Not Recommended', 'Recommended'], dtype=object)

In [55]:
def get_word_weights_for_class(classifier):
    """
    Get the word weights for a specific class in the classifier.
    :param classifier: The trained classifier pipeline.
    :param target_class: The target
    """
    vocabulary = classifier["vectorizer"].vocabulary_
    weights = classifier["log_regressor"].coef_
    print(weights.shape)

    word_weights = []
    for word in vocabulary.keys():
        # if number or too short, ignore
        if word.isnumeric() or len(word) < 3:
            continue
        j = vocabulary[word]
        word_weight_in_class = weights[0, j]
        word_weights.append((word_weight_in_class, word))
    
    return word_weights


In [56]:
# In this case (binary regressor) -> the coefficients that lead to dissatisfaction the most are the ones with the most negative value.
# And the ones that move away from dissatisfaction the most are the ones with the most positive value.
word_weights = get_word_weights_for_class(classifier)
target_str = classifier.classes_[0]

sorted_word_weights = sorted(word_weights, reverse=False, key=lambda x: x[0])
word_weights_sorted = [t[0] for t in sorted_word_weights]
words_sorted = [t[1] for t in sorted_word_weights]

# Separate top N negative and top N positive words
N_WORDS_TO_VISUALIZE = 30
negative_weights = word_weights_sorted[:N_WORDS_TO_VISUALIZE]
negative_words = words_sorted[:N_WORDS_TO_VISUALIZE]

positive_weights = word_weights_sorted[-N_WORDS_TO_VISUALIZE:]
positive_words = words_sorted[-N_WORDS_TO_VISUALIZE:]

# Create subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f"Words more related to '{target_str}'",
    f"Words less related to '{target_str}'"
])

# Negative bar chart
fig.add_trace(go.Bar(
    x=negative_weights,
    y=negative_words,
    orientation='h',
    marker=dict(color='crimson'),
    name='More correlated to the class'
), row=1, col=1)

# Positive bar chart
fig.add_trace(go.Bar(
    x=positive_weights,
    y=positive_words,
    orientation='h',
    marker=dict(color='seagreen'),
    name='Less correlated to the class'
), row=1, col=2)

# Update layout
fig.update_layout(
    height=600,
    width=1000,
    title_text=f"Correlation of words to the class '{target_str}'",
    showlegend=False,
    template="plotly_white"
)

# Improve axis labels
fig.update_yaxes(autorange="reversed", row=1, col=1)
fig.update_yaxes(row=1, col=2)

fig.show()


(1, 10325)


Here we can find some common points that the players dislike: like the crashes (and bugs), problems with perfomance (slow) and the game scripting (or DDA).

But we can find some things that seem to work like the career mode.

## Topic moddeling

Trying to visualize common topics on reviews.

In [69]:
vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=1000, stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 2)
)
X = vectorizer.fit_transform(not_satisfied["CleanedReview"])

N_TOPICS = 6
nmf = NMF(n_components=N_TOPICS, random_state=42)
nmf_W = nmf.fit_transform(X)
nmf_H = nmf.components_

feature_names = vectorizer.get_feature_names_out()

nmf_top = []
for topic_idx, topic_weights in enumerate(nmf_H):
    top_words = sorted(zip(feature_names, topic_weights), key=lambda x: -x[1])[:10]
    nmf_top.append((topic_idx, top_words))

In [70]:
fig = make_subplots(
    rows=3, cols=2, shared_yaxes=True,
    subplot_titles=[f"Topic {i+1}" for i in range(N_TOPICS)],
)

for i in range(N_TOPICS):
    topic = nmf_top[i][1]
    word = list(zip(*topic))[0]
    score = list(zip(*topic))[1]

    fig.add_trace(
        go.Bar(x=word, y=score, marker=dict(color=score, coloraxis="coloraxis")),
        row=i // 2 + 1, col=i % 2 + 1
    )

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False, height=800)
fig.show()


We can see some examples of reviews for each topic to make it easier to understand the topics.

In [71]:
not_satisfied_copy = not_satisfied.copy() # Create a copy to avoid modifying the original DataFrame
topic_assignments = nmf_W.argmax(axis=1)
not_satisfied_copy['topic'] = topic_assignments

for topic_id in range(N_TOPICS):
    print(f"\nTOPIC {topic_id + 1} EXAMPLE REVIEW:")
    sample = not_satisfied_copy[not_satisfied_copy['topic'] == topic_id]['Review'].iloc[0]
    print(sample)



TOPIC 1 EXAMPLE REVIEW:
Controller input is bugged, so can't play the game :)

TOPIC 2 EXAMPLE REVIEW:
Product refunded This game is actually amazing! It gives you a really outer body experience. You literally set low expectations and buy it for the sake of just playing FIFA after missing out on so many years.. and your body simply reacts automatically after you play for about 15 mins.. it just reacts and leads you to the refund page! Truly amazing. An outer body experience!

TOPIC 3 EXAMPLE REVIEW:
Year after year EA finds ways to make the game worse, especially for the PC players. Would not recommend getting it even on sales, feel sorry for those who paid the full price. Make this game free to play already, 0/10

TOPIC 4 EXAMPLE REVIEW:
if you are waiting to buy this game at a discount dont bother waiting. it isnt worth a single penny. worst game ive ever played. forget being the worst fifa game, its the worst game ive ever spent retail price for. ive been playing this franchise sin

We can get some insights over the topics with the help of the examples: 
- Problems with the inputs on controller (Topic 1). 
- Quality of the games of the FIFA franchise is decreasing over time (Topic 3). 
- Technical issues like, problems with the passing and shooting mechanics, and problems with the servers (Topic 6)   

### Topics for good reviews

In [63]:
vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=1000, stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 2)
)
X = vectorizer.fit_transform(satisfied["CleanedReview"])

N_TOPICS = 6
nmf = NMF(n_components=N_TOPICS, random_state=42)
nmf_W = nmf.fit_transform(X)
nmf_H = nmf.components_

feature_names = vectorizer.get_feature_names_out()

nmf_top = []
for topic_idx, topic_weights in enumerate(nmf_H):
    top_words = sorted(zip(feature_names, topic_weights), key=lambda x: -x[1])[:10]
    nmf_top.append((topic_idx, top_words))

fig = make_subplots(
    rows=3, cols=2, shared_yaxes=True,
    subplot_titles=[f"Topic {i+1}" for i in range(N_TOPICS)],
)

for i in range(N_TOPICS):
    topic = nmf_top[i][1]
    word = list(zip(*topic))[0]
    score = list(zip(*topic))[1]

    fig.add_trace(
        go.Bar(x=word, y=score, marker=dict(color=score, coloraxis="coloraxis")),
        row=i // 2 + 1, col=i % 2 + 1
    )

fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False, height=800)
fig.show()

In [64]:
satisfied_copy = satisfied.copy() # Create a copy to avoid modifying the original DataFrame
topic_assignments = nmf_W.argmax(axis=1)
satisfied_copy['topic'] = topic_assignments

for topic_id in range(N_TOPICS):
    print(f"\nTOPIC {topic_id + 1} EXAMPLE REVIEW:")
    sample = satisfied_copy[satisfied_copy['topic'] == topic_id]['Review'].iloc[0]
    print(sample)


TOPIC 1 EXAMPLE REVIEW:
If you're an Offline player like me who enjoys to play some Career modes then go for it, other than that Online modes are horrible as of today, this has the same release impact as FIFA 23, same problems. Career modes are fun and updated compared to last years game. It's only worth if you don't play Online much and when the price drop.

TOPIC 2 EXAMPLE REVIEW:
In my lifetime I have done some bad things. As part of self punishment I play EA Sports FC 25. I already feel like a changed man, this game puts into perspective how even game developers can do bad things like releasing this garbage game. Highly recommend if you are considering self harm at any point in your life, 10/10

TOPIC 3 EXAMPLE REVIEW:
I play only offline modes like manager and player career and other competitions. I've been staying away from ultimate team and other online modes that has been incurring complaints from many users. The game is good as far as my experience is concerned and is also th

With this topics we get some insights that confirm some previous hypothesis:

- With topic 1 we can confirm that the online seems to be a problem, but a lot of players like the career mode. Confirmed in Topic 3.

- With topic 4 we have a person that had the common problem with inputs (controler) but was able to fix on its own.

# Analysing most relevant comments with LLM

In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

For running this part, create a `.env` file in the same folder as the notebook and add the keys:
```
API_KEY = <Azure Open AI API KEY>
API_ENDPOINT = <Azure EndPoint>
```

In [2]:
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("API_ENDPOINT")
)

In [ ]:
top_10_reviews = satisfied.sort_values(by="Helpful", ascending=False).iloc[:10, :]["Review"].to_list()

review_list = "\n".join(f"- {i+1}: {r}" for i, r in enumerate(top_10_reviews))

good_reviews_prompt = f"""You are a helpful assistant. I will give you a list of reviews about a game.
Your task is to summarize the reviews, and search for insights that can be useful for the game developers.
Please, be very specific and detailed in your answer.
Here are the reviews:
{review_list}
"""

response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": good_reviews_prompt}],
)

out = response.choices[0].message.content

print(out)

with open("out/good_reviews_summary_gpt.md", "w") as f:
    f.write(out)

### **Summary of Reviews and Developer Insights for EA Sports FC 25**

#### **General Sentiments**
- **Improved but Still Flawed**: Most players agree that *EA Sports FC 25* is an improvement over previous titles like FIFA 23 and 24, particularly in offline gameplay and Career Mode. However, the game still suffers from issues that undermine the overall experience (e.g., bugs, copy-pasted elements, and pricing strategies).

- **Offline Gameplay is the Highlight**: Career Mode has garnered substantial praise, with users appreciating its updates, enhanced gameplay mechanics, and added realism. However, many feel these updates could still benefit from greater depth and refinement.

- **Online Modes Are Disappointing**: Online gameplay has received significant criticism for being repetitive, unbalanced, and plagued by unskilled players spamming meta strategies (e.g., overusing specific players like Mbappé). Many reviewers avoid online modes entirely due to these issues.

- **Technical Issue

In [ ]:
worse_10_reviews = not_satisfied.sort_values(by="Helpful", ascending=False).iloc[:10, :]["Review"].to_list()

review_list = "\n".join(f"- {i+1}: {r}" for i, r in enumerate(worse_10_reviews))

bad_reviews_prompt = f"""You are a helpful assistant. I will give you a list of only the bad reviews about a game.
Your task is to summarize the reviews, and search for insights that can be useful for the game developers.
Please, be very specific and detailed in your answer.
Here are the reviews:
{review_list}
"""

response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": bad_reviews_prompt}],
)

out = response.choices[0].message.content

print(out)

with open("out/bad_reviews_summary_gpt.md", "w") as f:
    f.write(out)

### Key Insights and Suggestions for Developers Based on Bad Reviews

The reviews of the game reveal a host of issues across gameplay, monetization, optimization, and general user experience. Below are detailed insights and possible avenues for improvement:

---

## 1. **Gameplay Mechanics: Lack of Responsiveness, AI Manipulation, and Perceived "Scripting"**
   Many players are frustrated with the in-game experience, particularly the mechanics that feel unfair, inconsistent, or outright "manipulated." Common issues include:
   - **Dynamic Difficulty Adjustment (DDA) or "Scripting":** Players feel that outcomes are artificially determined to drive monetary engagement in modes like Ultimate Team. Examples include:
     - AI inexplicably outperforming human players (e.g., low-rated defenders outrunning high-rated attackers).
     - Predictable match outcomes based on perceived AI intervention or “cheating.”
   - **Unresponsive Player Controls:**
     - Commands during crucial moments, suc

## Conclusion: FIFA 25 Review Analysis
The analysis of FIFA 25 reviews revealed several significant insights:

1. **Overall Sentiment**
   - The game received predominantly negative reviews
   - Many reviews had high helpfulness scores, indicating strong community agreement

2. **Key Issues Identified**
   - Gameplay mechanics and AI behavior were points of criticism
   - Technical issues including server problems, optimization and issues with the controller.
   - Concerns about microtransactions and pay-to-win elements
   - Frustration with the game's scripting and dynamic difficulty adjustment (DDA)

3. **Common Complaints**
   - Poor server performance and lag issues
   - Unbalanced gameplay mechanics
   - Aggressive monetization strategies
   - Lack of significant improvements from previous versions
   - Technical problems affecting PC players specifically

4. **Positive Aspects**
   - Some improvements in control mechanics
   - New features in manager mode
   - Return of pre-match animations
   - Rush mode received positive feedback

5. **Impact on Player Experience**
   - Many players expressed frustration with the game's current state
   - Technical issues significantly impacted gameplay enjoyment
   - Concerns about the game's direction and EA's handling of the franchise
   - Strong community feedback about the need for improvements

This analysis demonstrates the importance of addressing technical issues, gameplay balance, and monetization strategies in future iterations of the game. The high number of helpful votes on negative reviews suggests that these issues are widely recognized and experienced by the player community.